In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/test_y')
train_x, train_y, test_x, test_y

(dask.array<from-npy-stack, shape=(2400, 128, 128, 1), dtype=float32, chunksize=(2400, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(2400, 128, 128, 1), dtype=float32, chunksize=(2400, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(1600, 128, 128, 1), dtype=float32, chunksize=(1600, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(1600, 128, 128, 1), dtype=float32, chunksize=(1600, 128, 128, 1)>)

In [3]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(n_filters=16, dropout=0.25):
    image_input = tf.keras.Input(shape=(128, 128, 1), name='input_layer')
    
    conv_1 = build_block(image_input, n_filters)
    conv_2 = build_block(conv_1, n_filters)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)
    conv_4 = build_block(conv_3, n_filters * 2)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)
    conv_6 = build_block(conv_5, n_filters * 4)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)
    conv_8 = build_block(conv_7, n_filters * 8)
    pool_4 = tf.keras.layers.AveragePooling2D(padding='same')(conv_8)
    drop_4 = tf.keras.layers.SpatialDropout2D(dropout)(pool_4)
    
    conv_9 = build_block(drop_4, n_filters * 16)
    conv_10 = build_block(conv_9, n_filters * 16)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_10) #(-1, 16, 16, 256)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_8])#(-1, 16, 16, 384)
    upsp_1 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_1)#(-1, 16, 16, 384)
    conv_11 = build_block(upsp_1, n_filters * 8)#(-1, 16, 16, 128)
    conv_12 = build_block(conv_11, n_filters * 8)#(-1, 16, 16, 128)
    
    upsp_2 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_12) #(-1, 32, 32, 128)
    upsp_2 = tf.keras.layers.concatenate([upsp_2, conv_6]) #(-1, 32, 32, 192)
    upsp_2 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_2) #(-1, 32, 32, 192)
    conv_13 = build_block(upsp_2, n_filters * 4) #(-1, 32, 32, 64)
    conv_14 = build_block(conv_13, n_filters * 4) #(-1, 32, 32, 64)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_14) #(-1, 64, 64, 64)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 64, 64, 96)
    upsp_3 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_3) #(-1, 64, 64, 96)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 64, 64, 32)
    conv_16 = build_block(conv_15, n_filters * 2) #(-1, 64, 64, 32)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_16) #(-1, 128, 128, 32)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 128, 128, 48)
    upsp_4 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_4)#(-1, 128, 128, 48)
    conv_17 = build_block(upsp_4, n_filters)#(-1, 128, 128, 16)
    conv_18 = build_block(conv_17, n_filters)#(-1, 128, 128, 16)
    
    output = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='glorot_normal', activation='sigmoid')(conv_18)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model

In [5]:
model = build_unet()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 128, 128, 16) 144         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 128, 128, 16) 64          conv2d_19[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 128, 128, 16) 0           batch_normalization_18[0][0]     
__________________________________________________________________________________________________
conv2d_20 

In [6]:
epochs, batch_size, lr, filepath = 10000, 16, 0.001, "day_1.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True)
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = zip(image_generator, mask_generator)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=batch_size)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    #initial_epoch=65,
                    callbacks=[checkpoint, tb])

Epoch 1/10000
149/150 [============================>.] - ETA: 0s - loss: 9.7732 - acc: 0.6486
Epoch 00001: val_acc improved from -inf to 0.23850, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 59s 390ms/step - loss: 9.7528 - acc: 0.6500 - val_loss: 181.8590 - val_acc: 0.2385
Epoch 2/10000
149/150 [============================>.] - ETA: 0s - loss: 7.3100 - acc: 0.7899
Epoch 00002: val_acc improved from 0.23850 to 0.25449, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 28s 186ms/step - loss: 7.3149 - acc: 0.7896 - val_loss: 52.5210 - val_acc: 0.2545
Epoch 3/10000
149/150 [============================>.] - ETA: 0s - loss: 6.8603 - acc: 0.7916
Epoch 00003: val_acc improved from 0.25449 to 0.58067, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 28s 186ms/step - loss: 6.8612 - acc: 0.7916 - val_loss: 15.6353 - val_acc: 0.5807
Epoch 4/10000
149/150 [============================>.] - ETA

Epoch 30/10000
149/150 [============================>.] - ETA: 0s - loss: 4.9451 - acc: 0.8533
Epoch 00030: val_acc improved from 0.89782 to 0.90236, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 30s 199ms/step - loss: 4.9395 - acc: 0.8532 - val_loss: 3.8925 - val_acc: 0.9024
Epoch 31/10000
149/150 [============================>.] - ETA: 0s - loss: 4.7903 - acc: 0.8559
Epoch 00031: val_acc did not improve from 0.90236
150/150 [==============================] - 30s 198ms/step - loss: 4.7793 - acc: 0.8562 - val_loss: 4.4386 - val_acc: 0.8867
Epoch 32/10000
149/150 [============================>.] - ETA: 0s - loss: 4.7298 - acc: 0.8586
Epoch 00032: val_acc did not improve from 0.90236
150/150 [==============================] - 30s 198ms/step - loss: 4.7291 - acc: 0.8584 - val_loss: 4.0412 - val_acc: 0.8976
Epoch 33/10000
149/150 [============================>.] - ETA: 0s - loss: 4.7813 - acc: 0.8561
Epoch 00033: val_acc improved from 0.90236 to 0.90357

Epoch 60/10000
149/150 [============================>.] - ETA: 0s - loss: 4.2497 - acc: 0.8682
Epoch 00060: val_acc did not improve from 0.91375
150/150 [==============================] - 30s 198ms/step - loss: 4.2460 - acc: 0.8682 - val_loss: 3.3228 - val_acc: 0.9106
Epoch 61/10000
149/150 [============================>.] - ETA: 0s - loss: 4.0902 - acc: 0.8721
Epoch 00061: val_acc did not improve from 0.91375
150/150 [==============================] - 30s 198ms/step - loss: 4.0957 - acc: 0.8720 - val_loss: 3.3475 - val_acc: 0.9108
Epoch 62/10000
149/150 [============================>.] - ETA: 0s - loss: 4.1544 - acc: 0.8726
Epoch 00062: val_acc did not improve from 0.91375
150/150 [==============================] - 30s 198ms/step - loss: 4.1505 - acc: 0.8728 - val_loss: 3.4106 - val_acc: 0.9128
Epoch 63/10000
149/150 [============================>.] - ETA: 0s - loss: 4.1082 - acc: 0.8728
Epoch 00063: val_acc did not improve from 0.91375
150/150 [==============================] - 30s 1

Epoch 90/10000
149/150 [============================>.] - ETA: 0s - loss: 3.6565 - acc: 0.8829
Epoch 00090: val_acc improved from 0.91986 to 0.92358, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 30s 199ms/step - loss: 3.6494 - acc: 0.8829 - val_loss: 2.8902 - val_acc: 0.9236
Epoch 91/10000
149/150 [============================>.] - ETA: 0s - loss: 3.7099 - acc: 0.8828
Epoch 00091: val_acc did not improve from 0.92358
150/150 [==============================] - 30s 198ms/step - loss: 3.7269 - acc: 0.8824 - val_loss: 3.1809 - val_acc: 0.9116
Epoch 92/10000
149/150 [============================>.] - ETA: 0s - loss: 3.6561 - acc: 0.8830
Epoch 00092: val_acc did not improve from 0.92358
150/150 [==============================] - 30s 198ms/step - loss: 3.6448 - acc: 0.8834 - val_loss: 3.4678 - val_acc: 0.9048
Epoch 93/10000
149/150 [============================>.] - ETA: 0s - loss: 3.8640 - acc: 0.8790
Epoch 00093: val_acc did not improve from 0.92358
150

149/150 [============================>.] - ETA: 0s - loss: 3.3313 - acc: 0.8918
Epoch 00120: val_acc did not improve from 0.92429
150/150 [==============================] - 30s 198ms/step - loss: 3.3414 - acc: 0.8914 - val_loss: 2.7167 - val_acc: 0.9238
Epoch 121/10000
149/150 [============================>.] - ETA: 0s - loss: 3.5160 - acc: 0.8856
Epoch 00121: val_acc did not improve from 0.92429
150/150 [==============================] - 30s 198ms/step - loss: 3.5142 - acc: 0.8856 - val_loss: 3.2638 - val_acc: 0.9101
Epoch 122/10000
149/150 [============================>.] - ETA: 0s - loss: 3.4416 - acc: 0.8887
Epoch 00122: val_acc improved from 0.92429 to 0.92658, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 30s 199ms/step - loss: 3.4335 - acc: 0.8889 - val_loss: 2.7537 - val_acc: 0.9266
Epoch 123/10000
149/150 [============================>.] - ETA: 0s - loss: 3.4041 - acc: 0.8900
Epoch 00123: val_acc did not improve from 0.92658
150/150 [======

...

149/150 [============================>.] - ETA: 0s - loss: 1.3018 - acc: 0.9378
Epoch 01238: val_acc did not improve from 0.94765
150/150 [==============================] - 30s 197ms/step - loss: 1.3027 - acc: 0.9377 - val_loss: 2.3246 - val_acc: 0.9407
Epoch 1239/10000
149/150 [============================>.] - ETA: 0s - loss: 1.3335 - acc: 0.9375
Epoch 01239: val_acc did not improve from 0.94765
150/150 [==============================] - 30s 199ms/step - loss: 1.3328 - acc: 0.9374 - val_loss: 2.2422 - val_acc: 0.9447
Epoch 1240/10000
149/150 [============================>.] - ETA: 0s - loss: 1.2351 - acc: 0.9402
Epoch 01240: val_acc did not improve from 0.94765
150/150 [==============================] - 30s 199ms/step - loss: 1.2342 - acc: 0.9401 - val_loss: 2.4715 - val_acc: 0.9451
Epoch 1241/10000
149/150 [============================>.] - ETA: 0s - loss: 1.2424 - acc: 0.9401
Epoch 01241: val_acc did not improve from 0.94765
150/150 [==============================] - 30s 199ms/step

149/150 [============================>.] - ETA: 0s - loss: 1.2915 - acc: 0.9381
Epoch 01268: val_acc did not improve from 0.94765
150/150 [==============================] - 29s 196ms/step - loss: 1.2907 - acc: 0.9382 - val_loss: 2.4277 - val_acc: 0.9400
Epoch 1269/10000
149/150 [============================>.] - ETA: 0s - loss: 1.3758 - acc: 0.9377
Epoch 01269: val_acc did not improve from 0.94765
150/150 [==============================] - 29s 196ms/step - loss: 1.3835 - acc: 0.9374 - val_loss: 2.2545 - val_acc: 0.9448
Epoch 1270/10000
149/150 [============================>.] - ETA: 0s - loss: 1.3045 - acc: 0.9382
Epoch 01270: val_acc did not improve from 0.94765
150/150 [==============================] - 29s 196ms/step - loss: 1.3000 - acc: 0.9383 - val_loss: 2.3592 - val_acc: 0.9413
Epoch 1271/10000
149/150 [============================>.] - ETA: 0s - loss: 1.3130 - acc: 0.9367
Epoch 01271: val_acc did not improve from 0.94765
150/150 [==============================] - 29s 196ms/step

149/150 [============================>.] - ETA: 0s - loss: 1.2366 - acc: 0.9402
Epoch 01298: val_acc did not improve from 0.94765
150/150 [==============================] - 29s 196ms/step - loss: 1.2376 - acc: 0.9402 - val_loss: 2.2990 - val_acc: 0.9435
Epoch 1299/10000
149/150 [============================>.] - ETA: 0s - loss: 1.2406 - acc: 0.9392
Epoch 01299: val_acc did not improve from 0.94765
150/150 [==============================] - 29s 196ms/step - loss: 1.2440 - acc: 0.9392 - val_loss: 2.3896 - val_acc: 0.9419
Epoch 1300/10000
149/150 [============================>.] - ETA: 0s - loss: 1.1667 - acc: 0.9408
Epoch 01300: val_acc did not improve from 0.94765
150/150 [==============================] - 29s 196ms/step - loss: 1.1883 - acc: 0.9406 - val_loss: 2.3965 - val_acc: 0.9447
Epoch 1301/10000
149/150 [============================>.] - ETA: 0s - loss: 1.2180 - acc: 0.9400
Epoch 01301: val_acc did not improve from 0.94765
150/150 [==============================] - 29s 196ms/step

149/150 [============================>.] - ETA: 0s - loss: 1.2495 - acc: 0.9394
Epoch 01328: val_acc did not improve from 0.94765
150/150 [==============================] - 31s 205ms/step - loss: 1.2462 - acc: 0.9395 - val_loss: 2.7120 - val_acc: 0.9395
Epoch 1329/10000
149/150 [============================>.] - ETA: 0s - loss: 1.2953 - acc: 0.9377
Epoch 01329: val_acc did not improve from 0.94765
150/150 [==============================] - 31s 203ms/step - loss: 1.2916 - acc: 0.9378 - val_loss: 2.4121 - val_acc: 0.9396
Epoch 1330/10000
149/150 [============================>.] - ETA: 0s - loss: 1.1624 - acc: 0.9420
Epoch 01330: val_acc did not improve from 0.94765
150/150 [==============================] - 30s 203ms/step - loss: 1.1663 - acc: 0.9419 - val_loss: 2.6682 - val_acc: 0.9447
Epoch 1331/10000
149/150 [============================>.] - ETA: 0s - loss: 1.3093 - acc: 0.9370
Epoch 01331: val_acc did not improve from 0.94765
150/150 [==============================] - 31s 204ms/step

KeyboardInterrupt: 